In [1]:
import pandas as pd
from datetime import datetime
from yahoo_fin import stock_info as si

# Read the CSV file
df = pd.read_csv('dataset/CompanyData.csv', names=['Ticker', 'Company', 'Sector'])

# Extract unique tickers
tickers = df['Ticker'].unique()

# Define the start and end dates for the historical data
end_date = datetime.now()
start_date = end_date.replace(year=end_date.year - 5)

# Iterate over each ticker
for ticker in tickers:
    try:
        # Retrieve the historical data
        data = si.get_data(ticker, start_date=start_date, end_date=end_date, interval="1mo")
    
        # Convert the data into the desired format
        data['Year'] = data.index.year
        data['Month'] = data.index.month
        data = data[['Year', 'Month', 'open', 'close', 'high', 'low', 'adjclose', 'volume']]
        
        # Save the data to a CSV file
        filename = f'{ticker}_monthly_data.csv'
        data.to_csv('dataset/Five_Year_Month_Data/'+filename, index=False)
    except:
        print(f'No data found for {ticker}')

KeyboardInterrupt: 

In [1]:
import pandas as pd
from datetime import datetime
from yahoo_fin import stock_info as si

# Read the CSV file
df = pd.read_csv('dataset/CompanyData.csv', names=['Ticker', 'Company', 'Sector'])

# Extract unique tickers
tickers = df['Ticker'].unique()

# Define the start and end dates for the historical data
end_date = datetime.now()
start_date = end_date.replace(year=end_date.year - 1)

# Iterate over each ticker
for ticker in tickers:
    try:
        # Retrieve the historical data
        data = si.get_data(ticker, start_date=start_date, end_date=end_date, interval="1d")
    
        # Convert the data into the desired format
        data['Year'] = data.index.year
        data['Month'] = data.index.month
        data['Day'] = data.index.day
        data = data[['Year', 'Month', 'Day', 'open', 'close', 'high', 'low', 'adjclose', 'volume']]
        
        # Save the data to a CSV file
        filename = f'{ticker}_daily_data.csv'
        data.to_csv('dataset/One_Year_Daily_Data/'+filename, index=False)
    except Exception as e:
        print(e)
        print(f'No data found for {ticker}')

{'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
No data found for Ticker
{'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
No data found for BRK.B
'timestamp'
No data found for BF.B


In [3]:
import os
import pandas as pd
import numpy as np

# Get the list of CSV files in the directory
directory = 'dataset/Five_Year_Month_Data'
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Iterate over each CSV file
for file in csv_files:
    # Read the CSV file
    filepath = os.path.join(directory, file)
    df = pd.read_csv(filepath)
    
    # Calculate log return
    df['log_return'] = np.log(df['close']) - np.log(df['close'].shift(1))
    
    # Calculate log volume
    df['log_volume'] = np.log(df['volume']) - np.log(df['volume'].shift(1))
    
    # Save the updated dataframe to the CSV file
    df.to_csv("dataset/Five_Year_Month_Data_Log/" + file, index=False)


In [11]:
from scipy.stats import pearsonr
import pandas as pd

# Calculate the correlation between log returns of two stocks
def correlation_return(ticker1, ticker2):
    # Read the CSV files
    df1 = pd.read_csv(f'dataset/Five_Year_Month_Data_Log/{ticker1}_monthly_data.csv')
    df1 = df1.fillna(0)
    df2 = pd.read_csv(f'dataset/Five_Year_Month_Data_Log/{ticker2}_monthly_data.csv')
    df2 = df2.fillna(0)

    # If the two dataframes have different number of rows, drop the extra rows from the larger dataframe from the top
    if df1.shape[0] > df2.shape[0]:
        df1 = df1.iloc[df1.shape[0] - df2.shape[0]:]
    elif df2.shape[0] > df1.shape[0]:
        df2 = df2.iloc[df2.shape[0] - df1.shape[0]:]

    # Calculate the correlation between the log returns
    corr, p = pearsonr(df1['log_return'], df2['log_return'])
    return corr, p

# Calculate the correlation between log volumes of two stocks
def correlation_volume(ticker1, ticker2):
    # Read the CSV files
    df1 = pd.read_csv(f'dataset/Five_Year_Month_Data_Log/{ticker1}_monthly_data.csv')
    df1 = df1.dropna(subset=['log_volume'])
    df2 = pd.read_csv(f'dataset/Five_Year_Month_Data_Log/{ticker2}_monthly_data.csv')
    df2 = df2.dropna(subset=['log_volume'])
    
    # Merge the two dataframes on the Year and Month columns
    df = pd.merge(df1, df2, on=['Year', 'Month'])
    
    # Calculate the correlation between the log volumes
    corr, p = pearsonr(df['log_volume_x'], df['log_volume_y'])
    return corr, p

# Read the CSV file
df = pd.read_csv('dataset/CompanyData.csv', names=['Ticker', 'Company', 'Sector'])
# Extract unique tickers
tickers = df['Ticker'].unique()
# Create an empty dataframe to store the correlation values
correlation_df = pd.DataFrame(index=tickers, columns=tickers)
p_df = pd.DataFrame(index=tickers, columns=tickers)

def calculate_correlation(ticker1, ticker2, correlation_df, p_df):
    if ticker1 in ['Ticker', 'BRK.B', 'BF.B'] or ticker2 in ['Ticker', 'BRK.B', 'BF.B']:
        return  # Skip certain tickers
    
    correlation, p = correlation_return(ticker1, ticker2)
    
    # Update the dataframes with the calculated values
    correlation_df.loc[ticker1, ticker2] = correlation
    p_df.loc[ticker1, ticker2] = p

# Iterate over each ticker pair and calculate the correlation
for ticker1 in tickers:
    for ticker2 in tickers:
        calculate_correlation(ticker1, ticker2, correlation_df, p_df)

# Save the correlation dataframe to a CSV file
correlation_df.to_csv('dataset/Correlation_Return.csv', index=True)
p_df.to_csv('dataset/P_Return.csv', index=True)

/home/codespace/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/codespace/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/codespace/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/codespace/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/codespace/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInput

In [13]:
import pandas as pd
# PMFG/TMFG

# Root (2 * (1 - correlation))


# Read the correlation dataframe from the CSV file
correlation_df = pd.read_csv('dataset/Correlation_Return.csv', index_col=0)

# Remove rows and columns with ticker names 'Ticker', 'BRK.B', 'BF.B'
correlation_df = correlation_df.drop(['Ticker', 'BRK.B', 'BF.B', 'VLTO'], axis=0)
correlation_df = correlation_df.drop(['Ticker', 'BRK.B', 'BF.B', 'VLTO'], axis=1)

# Calculate the distance values using the formula: Root(2 * (1 - correlation))
distance_df = (2 * (1 - correlation_df)) ** 0.5

# Save the distance dataframe to a new CSV file
distance_df.to_csv('dataset/Distance_Return.csv', index=True)


In [14]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Read the distance dataframe from the CSV file
distance_df = pd.read_csv('dataset/Distance_Return.csv', index_col=0)

# Create a graph object
G = nx.Graph()

# Add nodes to the graph
for node in distance_df.index:
    G.add_node(node)

# Add edges to the graph
for i in range(len(distance_df)):
    for j in range(i+1, len(distance_df)):
        node1 = distance_df.index[i]
        node2 = distance_df.columns[j]
        distance = distance_df.iloc[i, j]
        G.add_edge(node1, node2, weight=distance)

# Create the PMFG graph
pmfg = nx.planar_maximal_graph(G)

# Draw the PMFG graph
pos = nx.spring_layout(pmfg)
nx.draw(pmfg, pos, with_labels=True, node_color='lightblue', edge_color='gray', font_size=8, node_size=200)

# Show the graph
plt.show()


AttributeError: module 'networkx' has no attribute 'planar_maximal_graph'

In [1]:
import shutil

# Specify the path of the project folder
project_folder = '../Project'

# Specify the path and name of the output zip file
output_zip = 'project.zip'

# Create a zip file of the project folder
shutil.make_archive(output_zip, 'zip', project_folder)


'/workspaces/CSC591-ML-UAS/Project/project.zip.zip'